In [689]:
import io
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter, landscape
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, Image, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch

In [690]:
filepath= "20250328-PointLomaNazarene-1.csv"
game_df = pd.read_csv(filepath)
game_df

,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,Batter,...,SpinAxis3dSeamOrientationBallAngleHorizontalAmb3,SpinAxis3dSeamOrientationBallAngleVerticalAmb3,SpinAxis3dSeamOrientationBallXAmb3,SpinAxis3dSeamOrientationBallYAmb3,SpinAxis3dSeamOrientationBallZAmb3,SpinAxis3dSeamOrientationBallAngleHorizontalAmb4,SpinAxis3dSeamOrientationBallAngleVerticalAmb4,SpinAxis3dSeamOrientationBallXAmb4,SpinAxis3dSeamOrientationBallYAmb4,SpinAxis3dSeamOrientationBallZAmb4
0,1,2025-03-28,12:05:27.81,1,1,"Cebulski, Ray",10018397,Right,POI_LOM,"Mallonee, Trent",...,-2.931083,2.592638,-0.051082,0.045235,0.997669,-177.403971,2.928080,-0.045235,0.051082,-0.997669
1,2,2025-03-28,12:05:42.42,1,2,"Cebulski, Ray",10018397,Right,POI_LOM,"Mallonee, Trent",...,-1.049923,0.251609,-0.018323,0.004391,0.999822,-179.748349,1.049913,-0.004391,0.018323,-0.999822
2,3,2025-03-28,12:06:14.92,2,1,"Cebulski, Ray",10018397,Right,POI_LOM,"Henderson, Gabriel",...,-81.537117,2.256643,-0.988344,0.039376,0.147055,-165.009991,81.243532,-0.039376,0.988344,-0.147055
3,4,2025-03-28,12:06:29.32,2,2,"Cebulski, Ray",10018397,Right,POI_LOM,"Henderson, Gabriel",...,-8.041449,3.314542,-0.139655,0.057817,0.988511,-176.652617,8.027909,-0.057817,0.139655,-0.988511
4,5,2025-03-28,12:06:45.36,2,3,"Cebulski, Ray",10018397,Right,POI_LOM,"Henderson, Gabriel",...,-0.449081,6.316794,-0.007790,0.110026,0.993898,-173.683013,0.446354,-0.110026,0.007790,-0.993898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,401,2025-03-28,15:22:10.08,4,1,"Ruiz, Tim",1000324213,Right,FRE_PAC,"Bernath, Matthew",...,-1.634243,-42.936642,-0.020879,-0.681189,0.731810,-137.051734,-1.196366,-0.681189,-0.020879,-0.731810
401,402,2025-03-28,15:22:27.09,4,2,"Ruiz, Tim",1000324213,Right,FRE_PAC,"Bernath, Matthew",...,-17.988086,5.036708,-0.307627,0.087794,0.947448,-174.705885,17.916266,-0.087794,0.307627,-0.947448
402,403,2025-03-28,15:22:48.28,4,3,"Ruiz, Tim",1000324213,Right,FRE_PAC,"Bernath, Matthew",...,-23.894794,3.876967,-0.404132,0.067614,0.912198,-175.760860,23.836718,-0.067614,0.404132,-0.912198
403,404,2025-03-28,15:23:14.25,4,4,"Ruiz, Tim",1000324213,Right,FRE_PAC,"Bernath, Matthew",...,-0.216245,45.335751,-0.002653,0.711238,0.702946,-134.664045,0.152010,-0.711238,0.002653,-0.702946


In [691]:
point_loma_pitchers = game_df[game_df['PitcherTeam'] == 'POI_LOM']['Pitcher'].unique()
print(point_loma_pitchers)

['Cebulski, Ray' 'Brown, Bobby' 'Williams, Christian']


In [692]:
pitcherName = 'Cebulski, Ray'

In [693]:
df = game_df[game_df['Pitcher'] == pitcherName]

In [694]:
total_pitches = len(df)
pitch_usage = (df['TaggedPitchType'].value_counts() / total_pitches * 100).round(2)

print("Pitch Usage Breakdown:")
print("-" * 30)
for pitch_type, usage in pitch_usage.items():
    print(f"{pitch_type}: {usage}%")

Pitch Usage Breakdown:
------------------------------
Fastball: 45.54%
Sinker: 20.54%
ChangeUp: 16.96%
Slider: 11.61%
Curveball: 3.57%
Sweeper: 1.79%


In [695]:
def countPitches(pitch_type):
    pitches = len(df[df['TaggedPitchType'] == pitch_type])
    return pitches

In [696]:
def calculateUsePercentage(pitch_type):
    pitches = len(df[df['TaggedPitchType'] == pitch_type])
    return round((pitches / total_pitches*100),1)

In [697]:
def calculateStrikePercentage(pitch_type):
    pitches = len(df[df['TaggedPitchType'] == pitch_type])
    strikes = len(df[(df['TaggedPitchType'] == pitch_type) & (df['PitchCall'].isin(['StrikeSwinging', 'StrikeCalled', 'FoulBallNotFieldable', 'FoulBallFieldable', 'InPlay']))])
    return round(((strikes/pitches)*100 if pitches > 0 else 0),1)

In [698]:
def calculateCSW(pitch_type):
    pitches = len(df[df['TaggedPitchType'] == pitch_type])
    strikes = len(df[(df['TaggedPitchType'] == pitch_type) & (df['PitchCall'].isin(['StrikeSwinging', 'StrikeCalled']))])
    return round(((strikes/pitches)*100 if pitches > 0 else 0),1)

In [699]:
def calculateMaxVelocity(pitch_type):
    maxVelo = df[df['TaggedPitchType'] == pitch_type]['RelSpeed'].max()
    return round(maxVelo,2)

In [700]:
def calculateAverageVelocity(pitch_type):
    avgVelo = df[df['TaggedPitchType'] == pitch_type]['RelSpeed'].mean()
    return round(avgVelo,2)

In [701]:
def calculateSpinRate(pitch_type):
    avg_spin = df[df['TaggedPitchType'] == pitch_type]['SpinRate'].mean()
    return round(avg_spin,0)

In [702]:
def calculateHorzBreak(pitch_type):
    hbreak = df[df['TaggedPitchType'] == pitch_type]['HorzBreak'].mean()
    return round(hbreak,2)

In [703]:
def calculateVertBreak(pitch_type):
    hbreak = df[df['TaggedPitchType'] == pitch_type]['InducedVertBreak'].mean()
    return round(hbreak,2)

In [704]:
def calculateExitVelo(pitch_type):
    filtered_df = df[(df['TaggedPitchType'] == pitch_type) & (df['PitchCall'] == 'InPlay')]
    velo = filtered_df['ExitSpeed'].mean()
    return "NA" if pd.isna(velo) else round(velo, 2)

In [705]:
def calculateHardHit(pitch_type):
    in_play_df = df[(df['TaggedPitchType'] == pitch_type) & (df['PitchCall'] == 'InPlay')]
    total_hits = len(in_play_df[in_play_df['ExitSpeed'].notna()])
    hard_hits = len(in_play_df[in_play_df['ExitSpeed'] > 90])
    
    if total_hits == 0:
        return "NA"  # Return NaN if no balls were in play
    return round((hard_hits / total_hits) * 100, 1)

In [706]:
def calculateStuffPlus(pitch_type):
    # Load the league-wide reference stats
    league_stats = pd.read_csv("league_reference_stats.csv")

    # Ensure required columns exist
    required_columns = ["Pitcher", "TaggedPitchType", "RelSpeed", "SpinRate", 
                        "InducedVertBreak", "HorzBreak", "Extension", "RelHeight"]
    
    if not all(col in game_df.columns for col in required_columns):
        raise ValueError("The game dataset does not contain all required columns.")
    
    game_df_copy = game_df

    # Drop rows with missing data in required columns
    game_df_copy = game_df_copy[required_columns].dropna()

    # Merge league stats onto game data based on TaggedPitchType
    game_df_copy = game_df_copy.merge(league_stats, on="TaggedPitchType", how="left")

    # Compute z-scores for each metric using the league-wide mean and std
    for feature in ["RelSpeed", "SpinRate", "InducedVertBreak", "HorzBreak", "Extension", "RelHeight"]:
        game_df_copy[f"{feature}_z"] = (game_df_copy[feature] - game_df_copy[f"{feature}_mean"]) / game_df_copy[f"{feature}_std"]

    # Compute Stuff_Score using weighted formula
    game_df_copy["Stuff_Score"] = (
        0.4  * game_df_copy["RelSpeed_z"] +
        0.3  * game_df_copy["SpinRate_z"] +
        0.2  * game_df_copy["InducedVertBreak_z"] +
        0.15 * game_df_copy["HorzBreak_z"] +
        0.1  * game_df_copy["Extension_z"] +
        0.05 * game_df_copy["RelHeight_z"]
    )

    # Scale Stuff+ so that the league average is 100
    mean_raw = game_df_copy["Stuff_Score"].mean()
    game_df_copy["Stuff+"] = 100 + (game_df_copy["Stuff_Score"] - mean_raw) * 10

    # Filter for the specific pitcher and pitch type
    filtered_df = game_df_copy[
        (game_df_copy["Pitcher"] == pitcherName) & 
        (game_df_copy["TaggedPitchType"] == pitch_type)
    ]

    # If no data found, return None
    if filtered_df.empty:
        return None

    # Compute the average Stuff+ for the pitcher and pitch type
    return filtered_df["Stuff+"].mean()

In [707]:
pitch_stats = {}
unique_pitches = df['TaggedPitchType'].unique()

In [708]:
for pitch in unique_pitches:
    pitch_stats[pitch] = {
        'total_pitches': countPitches(pitch),
        "use_percentage": calculateUsePercentage(pitch),
        'strike_percentage': calculateStrikePercentage(pitch),
        'csw_percentage': calculateCSW(pitch),
        'max_velo': calculateMaxVelocity(pitch),
        'avg_vel': calculateAverageVelocity(pitch),
        'avg_spin': calculateSpinRate(pitch),
        'horz_break': calculateHorzBreak(pitch),
        'vert_break': calculateVertBreak(pitch),
        'exit_speed': calculateExitVelo(pitch),
        'hard_hit_percentage': calculateHardHit(pitch),
        'stuff+': calculateStuffPlus(pitch)
    }

pitch_stats = dict(sorted(pitch_stats.items(), key=lambda item: item[1]['total_pitches'], reverse=True))

In [709]:
df_stats = pd.DataFrame(pitch_stats).T
df_stats.index.name = 'Pitch Type'
df_stats

,total_pitches,use_percentage,strike_percentage,csw_percentage,max_velo,avg_vel,avg_spin,horz_break,vert_break,exit_speed,hard_hit_percentage,stuff+
Pitch Type,,,,,,,,,,,,
Fastball,51.0,45.5,58.8,21.6,90.53,87.72,2101.0,13.3,15.47,89.22,55.6,99.269371
Sinker,23.0,20.5,65.2,21.7,89.58,87.35,2007.0,16.37,9.57,82.56,57.1,98.040209
ChangeUp,19.0,17.0,57.9,10.5,80.18,78.72,1224.0,14.03,1.99,78.08,0.0,93.920053
Slider,13.0,11.6,76.9,38.5,83.13,80.25,2245.0,-4.78,3.39,75.54,0.0,102.659684
Curveball,4.0,3.6,25.0,0.0,77.02,75.65,2602.0,-13.14,-8.15,86.94,0.0,103.385649
Sweeper,2,1.8,100.0,50.0,76.97,76.66,2470.0,-15.15,-1.04,NA,NA,99.696713


In [710]:
def calculateInningsPitched():
    outs = df['OutsOnPlay'].sum() + len(df[df['KorBB'] == 'Strikeout'])
    innings = round(outs/3,0)
    decimal = outs%3

    innings += decimal /10
    return innings

In [711]:
def calculatePitchesThrown():
    return len(df)

In [712]:
def calculateBattersFaced():
    return len(df.groupby(['Inning', 'PAofInning', 'GameID']))

In [713]:
def calculateStrikePercentageTotal():
    strikes = len(df[df['PitchCall'].isin(['StrikeSwinging', 'StrikeCalled', 'FoulBallNotFieldable', 'FoulBallFieldable', 'InPlay'])])
    return round((strikes/len(df))*100, 1)

In [714]:
def calculateTotalStrikeouts():
    return len(df[df['KorBB'] == 'Strikeout'])

In [715]:
def calculateStrikeoutPercentage():
    batters = calculateBattersFaced()
    return round((calculateTotalStrikeouts()/batters)*100, 1) if batters > 0 else 0

In [716]:
def calculateTotalWalks():
    return len(df[df['KorBB'] == 'Walk'])

In [717]:
def calaulateWalkPercentage():
    batters = calculateBattersFaced()
    return round((calculateTotalWalks()/batters)*100, 1) if batters > 0 else 0

In [718]:
def calculateCSWPercentage():
    csw = len(df[df['PitchCall'].isin(['StrikeSwinging', 'StrikeCalled'])])
    return round((csw/len(df))*100, 1)

In [719]:
def calculateFPS():
    first_pitches = df[df['PitchofPA'] == 1]
    strikes = first_pitches[first_pitches['PitchCall'].isin(['StrikeSwinging', 'StrikeCalled', 'FoulBallNotFieldable', 'FoulBallFieldable', 'InPlay'])]
    return len(strikes)

In [720]:
def calculateFPSPercentage():
    first_pitches = df[df['PitchofPA'] == 1]
    strikes = first_pitches[first_pitches['PitchCall'].isin(['StrikeSwinging', 'StrikeCalled', 'FoulBallNotFieldable', 'FoulBallFieldable', 'InPlay'])]
    return round((len(strikes)/len(first_pitches))*100, 1)

In [721]:
def calculateTotalHits():
    return len(df[df['PlayResult'].isin(['Single', 'Double', 'Triple', 'HomeRun'])])

In [722]:
game_stats = {
        'IP': calculateInningsPitched(),
        'Pitches': calculatePitchesThrown(),
        'BF': calculateBattersFaced(),
        'Strike%': calculateStrikePercentageTotal(),
        'K': calculateTotalStrikeouts(),
        'K%': calculateStrikeoutPercentage(),
        'BB': calculateTotalWalks(),
        'BB%': calaulateWalkPercentage(),
        'CSW%': calculateCSWPercentage(),
        'FPS': calculateFPS(),
        'FPS%': calculateFPSPercentage(),
        'H': calculateTotalHits()
}

In [723]:
statLine = pd.DataFrame([game_stats],index=[''])
statLine

,IP,Pitches,BF,Strike%,K,K%,BB,BB%,CSW%,FPS,FPS%,H
,6.0,112,30,61.6,2,6.7,3,10.0,21.4,19,63.3,9


In [724]:
# Define PLNU colors
PLNU_GREEN = colors.HexColor('#006B54')  # PLNU Dark Green
PLNU_GOLD = colors.HexColor('#CBA052')   # PLNU Gold

In [725]:
# Define pitch type colors with specific hex codes
pitch_colors = {
    "Fastball": "#FF4040",  # Bright red
    "Sinker": "#FF8C00",    # Dark orange
    "ChangeUp": "#32CD32",  # Lime green
    "Slider": "#9370DB",    # Medium purple
    "Sweeper": "#FFD700",   # Gold
    "Curveball": "#4169E1", # Royal blue
    "Cutter": "#363636"     # Dark gray
}

In [726]:
def draw_strike_zone(ax):
    strike_zone_width = (17+2.9)/12  # plate + ball
    strike_zone_height = [15/12, 43/12] # 6'
    buffer = 2.9/12

    # Dashed standard strike zone (17 inches wide)
    standard_zone = plt.Rectangle(
        (-strike_zone_width / 2, strike_zone_height[0]),
        strike_zone_width,
        strike_zone_height[1] - strike_zone_height[0],
        fill=False,
        color='black',
        linestyle='-',
        linewidth=2
    )
    ax.add_patch(standard_zone)

    # Solid extended strike zone (with ball buffer)
    extended_zone = plt.Rectangle(
        (-strike_zone_width / 2 - buffer, strike_zone_height[0] - buffer),
        strike_zone_width + 2 * buffer,
        (strike_zone_height[1] - strike_zone_height[0]) + 2 * buffer,
        fill=False,
        color='black',
        linestyle='--',
        linewidth=2
    )
    ax.add_patch(extended_zone)

    # Home plate line
    ax.plot([-(strike_zone_width-buffer) / 2, (strike_zone_width-buffer) / 2], [0, 0], 'k-', linewidth=2)

In [727]:
def save_plt_as_png(fig):
    """Saves a matplotlib figure as a PNG image in memory."""
    img_data = io.BytesIO()
    fig.savefig(img_data, format='png', bbox_inches='tight')
    img_data.seek(0)
    return img_data

In [728]:
def create_pitch_location(df):
    
    fig, ax = plt.subplots(figsize=(6, 6))
    
    if not df.empty:
        pitch_counts = df["TaggedPitchType"].value_counts()
        sorted_pitches = pitch_counts.index.tolist()

        for pitch_type in sorted_pitches:
            subset = df[df["TaggedPitchType"] == pitch_type]
            ax.scatter(subset["PlateLocSide"], subset["PlateLocHeight"], 
                       label=pitch_type, alpha=0.8, s=144,
                       color=pitch_colors.get(pitch_type, "gray"))

        draw_strike_zone(ax)  # Add strike zone

        ax.set_title("Pitch Location (Pitcher's Perspective)", fontsize=16)
        ax.set_xlabel("Horizontal Location (ft)", fontsize=14)
        ax.set_ylabel("Height (ft)", fontsize=14)
        ax.set_xlim(-3, 3)
        ax.set_ylim(-1, 6)
        ax.grid(True, alpha=0.3)
        ax.tick_params(axis='both', which='major', labelsize=10)
    plt.tight_layout()
    plt.close(fig)
    return fig

In [729]:
def create_release_point(df):
    fig, ax = plt.subplots(figsize=(6, 6))

    if not df.empty:
        pitch_counts = df["TaggedPitchType"].value_counts()
        sorted_pitches = pitch_counts.index.tolist()

        for pitch_type in sorted_pitches:
            subset = df[df["TaggedPitchType"] == pitch_type]
            ax.scatter(subset["RelSide"], subset["RelHeight"], 
                       label=pitch_type, alpha=0.8, s=80,
                       color=pitch_colors.get(pitch_type, "gray"))

        # Add the pitcher's mound and rubber
        mound_flat = plt.Rectangle((-2.5, 0), 5, 0.833, fill=True, color="saddlebrown", alpha=0.6)
        mound_slope = plt.Polygon([[-3.5, 0], [-2.5, 0.833], [2.5, 0.833], [3.5, 0]], closed=True, color="saddlebrown", alpha=0.6)
        rubber = plt.Rectangle((-1, 0.833), 2, 0.05, fill=True, color="black", alpha=1.0)

        ax.add_patch(mound_flat)
        ax.add_patch(mound_slope)
        ax.add_patch(rubber)

        ax.set_title("Release Point (Pitcher's Perspective)", fontsize=16)
        ax.set_xlabel("Release Side (ft)", fontsize=14)
        ax.set_ylabel("Release Height (ft)", fontsize=14)
        ax.set_xlim(-4, 4)
        ax.set_ylim(0, 7)
        ax.grid(True, alpha=0.3)
        ax.tick_params(axis='both', which='major', labelsize=10)
    plt.tight_layout()
    plt.close(fig)
    return fig

In [730]:
def create_pitch_movement(df):
    fig, ax = plt.subplots(figsize=(6, 6))

    if not df.empty:
        pitch_counts = df["TaggedPitchType"].value_counts()
        sorted_pitches = pitch_counts.index.tolist()

        for pitch_type in sorted_pitches:
            subset = df[df["TaggedPitchType"] == pitch_type]
            ax.scatter(subset["HorzBreak"], subset["InducedVertBreak"],
                       label=pitch_type, alpha=0.8, s=60,
                       color=pitch_colors.get(pitch_type, "gray"))

        ax.set_title("Pitch Movement", fontsize=16)
        ax.set_xlabel("Horizontal Break (in)", fontsize=14)
        ax.set_ylabel("Induced Vertical Break (in)", fontsize=14)
        ax.set_xlim(-30, 30)
        ax.set_ylim(-20, 30)
        ax.axhline(0, color='black', linewidth=0.5, linestyle='dashed')
        ax.axvline(0, color='black', linewidth=0.5, linestyle='dashed')
        ax.grid(True)
        ax.tick_params(axis='both', which='major', labelsize=10)
    plt.tight_layout()
    plt.close(fig)
    return fig

In [731]:
def create_velo_inning(df):
    fig, ax = plt.subplots(figsize=(6, 6))

    if not df.empty:
        avg_velocities = df.groupby("TaggedPitchType")["RelSpeed"].mean().sort_values(ascending=False)
        avg_velocity_by_inning = df.groupby(["Inning", "TaggedPitchType"])["RelSpeed"].mean().reset_index()

        for pitch_type in avg_velocities.index:
            subset = avg_velocity_by_inning[avg_velocity_by_inning["TaggedPitchType"] == pitch_type]
            ax.plot(subset["Inning"], subset["RelSpeed"],
                    marker="o", linestyle="-", label=pitch_type,
                    linewidth=2, alpha=0.8,
                    color=pitch_colors.get(pitch_type, "gray"))

        ax.set_title("Velocity by Inning", fontsize=16)
        ax.set_xlabel("Inning", fontsize=14)
        ax.set_ylabel("Average Release Speed (mph)", fontsize=14)
        ax.set_xticks(sorted(avg_velocity_by_inning["Inning"].unique()))
        ax.grid(True, linestyle="--", alpha=0.6)
        ax.tick_params(axis='both', which='major', labelsize=10)

    plt.tight_layout()
    plt.close(fig)
    return fig

In [732]:
def create_batted_heatmap(df):
    # Filter for only balls put in play
    batter_data = df[
        df['PitchCall'] == 'InPlay'
    ].dropna(subset=['PlateLocSide', 'PlateLocHeight'])

    if batter_data.empty or len(batter_data) < 3:
        return None

    # Create the density heatmap
    fig, ax = plt.subplots(figsize=(6, 6))

    sns.kdeplot(
        data=batter_data,
        x='PlateLocSide',
        y='PlateLocHeight',
        fill=True,
        cmap='RdBu_r',
        ax=ax
    )

    draw_strike_zone(ax)

    # Formatting
    ax.set_title('Balls In Play', fontsize=18)
    ax.set_xlabel('Horizontal Location (ft)', fontsize=14)
    ax.set_ylabel("Height (ft)", fontsize=14)
    ax.set_xlim(3, -3)
    ax.set_ylim(-1, 6)
    ax.tick_params(axis='both', which='major', labelsize=10)

    plt.tight_layout()
    plt.close(fig)
    return fig

In [733]:
def create_heatmaps(df):
    pitcher_data = df.copy()
    pitcher_data = pitcher_data.dropna(subset=['PlateLocSide', 'PlateLocHeight', 'TaggedPitchType'])

    if pitcher_data.empty:
        return None

    # Filter dataset for two-strike count pitches (0-2, 1-2, 2-2)
    two_strike_data = pitcher_data[
        (pitcher_data["Strikes"] == 2) & (pitcher_data["Balls"].isin([0, 1, 2]))
    ]

    # Include two-strike heatmap only if it has at least 3 pitches
    include_two_strike = len(two_strike_data) >= 3

    # Count pitches and filter out pitch types with fewer than 3 pitches
    pitch_counts = pitcher_data['TaggedPitchType'].value_counts()
    valid_pitch_types = pitch_counts[pitch_counts >= 3].index.tolist()

    # Total number of heatmaps
    n_pitches = len(valid_pitch_types) + (1 if include_two_strike else 0)

    if n_pitches == 0:
        return None

    # Create subplots
    fig, axes = plt.subplots(1, n_pitches, figsize=(6 * n_pitches, 6), squeeze=False)
    axes = axes.flatten()
    idx = 0

    # Plot two-strike count heatmap first, if applicable
    if include_two_strike:
        ax = axes[idx]
        sns.kdeplot(
            data=two_strike_data,
            x='PlateLocSide',
            y='PlateLocHeight',
            fill=True,
            cmap='RdBu_r',
            ax=ax
        )
        draw_strike_zone(ax)
        ax.set_title('Two-Strike Counts', fontsize=18)
        ax.set_xlabel('Horizontal Location (ft)', fontsize=15)
        ax.set_ylabel('Height (ft)', fontsize=15)
        ax.set_xlim(-3, 3)
        ax.set_ylim(-1, 5)
        ax.tick_params(axis='both', which='major', labelsize=10)
        idx += 1

    # Plot pitch type heatmaps in order of frequency
    for pitch_type in valid_pitch_types:
        pitch_data = pitcher_data[pitcher_data['TaggedPitchType'] == pitch_type]
        ax = axes[idx]
        sns.kdeplot(
            data=pitch_data,
            x='PlateLocSide',
            y='PlateLocHeight',
            fill=True,
            cmap='RdBu_r',
            ax=ax
        )
        draw_strike_zone(ax)
        ax.set_title(f'{pitch_type}', fontsize=18)
        ax.set_xlabel('Horizontal Location (ft)', fontsize=15)
        ax.set_ylabel('Height (ft)', fontsize=15)
        ax.set_xlim(-3, 3)
        ax.set_ylim(-1, 5)
        ax.tick_params(axis='both', which='major', labelsize=10)
        idx += 1

    plt.tight_layout()
    plt.close(fig)
    return fig

In [734]:
def create_pitcher_report(pitcher_name, pitch_stats_df, game_stats_df, df):

    # Find most common BatterTeam
    top_team = game_df[game_df['BatterTeam'] != 'POI_LOM']['BatterTeam'].mode().iloc[0]
    safe_team = top_team.replace(" ", "_")

    # Build the path: Reports/<OpponentTeam>/
    reports_folder = os.path.join("Reports")

    # Ensure the team-specific folder exists
    os.makedirs(reports_folder, exist_ok=True)

    # Clean and reorder pitcher name from "Lastname, Firstname" to "FirstnameLastname"
    last, first = pitcher_name.split(", ")
    safe_pitcher_name = f"{first}{last}".replace(" ", "")

    # Create filename
    output_filename = os.path.join(reports_folder, f"{safe_pitcher_name}_vs_{safe_team}.pdf")

    # Create document with landscape orientation
    doc = SimpleDocTemplate(
        output_filename,
        pagesize=landscape(letter),
        rightMargin=10,
        leftMargin=10,
        topMargin=10,
        bottomMargin=10
    )

    # Container for the 'Flowables'
    elements = []

    # Get styles
    styles = getSampleStyleSheet()
    title_style = ParagraphStyle(
        'CustomTitle',
        parent=styles['Heading1'],
        fontSize=24,
        spaceAfter=7.5,
        alignment=1,
        textColor=PLNU_GREEN
    )

    heading_style = ParagraphStyle(
        'CustomHeading',
        parent=styles['Heading2'],
        textColor=PLNU_GREEN
    )

    # Convert "Last, First" to "First Last" for display
    if ", " in pitcher_name:
        last_name, first_name = pitcher_name.split(", ")
        display_pitcher_name = f"{first_name} {last_name}"
    else:
        display_pitcher_name = pitcher_name  # If already formatted correctly

    # Add title
    elements.append(Paragraph("Point Loma Nazarene University", title_style))
    elements.append(Paragraph(f"Pitcher Analysis Report: {display_pitcher_name}", title_style))
    elements.append(Spacer(1, 1))

    # Add pitch statistics section
    elements.append(Paragraph("Pitch Statistics", heading_style))
    elements.append(Spacer(1, 1))

    # Rename columns
    pitch_stats_df = pitch_stats_df.rename(columns={
        'total_pitches': 'Total',
        'use_percentage': 'Usage%',
        'strike_percentage': 'Strike%',
        'csw_percentage': 'CSW%',
        'max_velo': 'MaxVelo',
        'avg_vel': 'AvgVelo',
        'avg_spin': 'Spin',
        'horz_break': 'HBreak',
        'vert_break': 'VBreak',
        'exit_speed': 'ExitVelo',
        'hard_hit_percentage': 'HardHit%',
        'stuff+': 'Stuff+'
    })

    pitch_data = [['Pitch'] + list(pitch_stats_df.columns)]
    for idx, row in pitch_stats_df.iterrows():
        pitch_data.append([idx] + [
            str(int(x)) if col in ['Total', 'Spin'] and isinstance(x, (int, float)) else 
            f"{float(x):.1f}" if col in ['Usage%', 'Strike%', 'CSW%', 'HardHit%', 'Stuff+'] and isinstance(x, (int, float)) else 
            f"{float(x):.2f}" if col in ['MaxVelo', 'AvgVelo', 'ExitVelo', 'HBreak', 'VBreak', 'ExitVelo'] and isinstance(x, (int, float)) else 
            str(x)  # Default case
            for col, x in row.items()
        ])

    # Calculate column widths
    col_widths = [1.2 * inch]
    num_cols = len(pitch_data[0])
    available_width = doc.width - col_widths[0]
    col_widths.extend([available_width / (num_cols - 1)] * (num_cols - 1))

    # Create pitch stats table
    pitch_table = Table(pitch_data, colWidths=col_widths)
    pitch_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), PLNU_GREEN),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 9),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.white),
        ('TEXTCOLOR', (0, 1), (-1, -1), colors.black),
        ('FONTNAME', (0, 1), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 1), (-1, -1), 9),
        ('GRID', (0, 0), (-1, -1), 1, PLNU_GREEN),
        ('BOX', (0, 0), (-1, -1), 2, PLNU_GREEN),
        ('LINEBELOW', (0, 0), (-1, 0), 2, PLNU_GOLD),
    ]))
    elements.append(pitch_table)
    elements.append(Spacer(1, 5))

    # Add game statistics section
    elements.append(Paragraph("Game Statistics", heading_style))
    elements.append(Spacer(1, 5))

    # Round only the specified columns in game statistics
    for col in ['pitches', 'BF', 'K', 'BB', 'FPS', 'hits']:
        if col in game_stats_df.columns:
            game_stats_df[col] = game_stats_df[col].round(0).astype(int)

    # Convert game stats DataFrame to table data
    game_data = [list(game_stats_df.columns)]
    game_data.extend(game_stats_df.astype(str).values.tolist())

    # Calculate column widths for game stats table
    game_col_widths = [doc.width / len(game_data[0])] * len(game_data[0])

    # Create game stats table
    game_table = Table(game_data, colWidths=game_col_widths)
    game_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), PLNU_GREEN),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 10),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.white),
        ('TEXTCOLOR', (0, 1), (-1, -1), colors.black),
        ('FONTNAME', (0, 1), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 1), (-1, -1), 9),
        ('GRID', (0, 0), (-1, -1), 1, PLNU_GREEN),
        ('BOX', (0, 0), (-1, -1), 2, PLNU_GREEN),
        ('LINEBELOW', (0, 0), (-1, 0), 2, PLNU_GOLD),
    ]))
    elements.append(game_table)

    # ---- Heatmaps (Now on Page 1, below game statistics) ----
    elements.append(Paragraph("Pitch Heatmaps (Pitcher's Perspective)", heading_style))
    elements.append(Spacer(1, 5))

    # Generate heatmaps
    heatmap_fig = create_heatmaps(df)

     # Heatmap image (resized for 2x3 layout)
    if heatmap_fig:
        heatmap_img = Image(save_plt_as_png(heatmap_fig), width=10.75 * inch, height=2.75 * inch)
    else:
        heatmap_img = None  # Ensure the report doesn't fail if no heatmaps exist

    elements.append(heatmap_img)

    # ---- Page 2: Scatter Plots (2x2 Layout) ----
    elements.append(PageBreak())
    elements.append(Paragraph("Pitch Visualizations", heading_style))
    elements.append(Spacer(1, 1))

    top_left_fig = create_pitch_location(df)
    top_right_fig = create_release_point(df)
    bottom_left_fig = create_pitch_movement(df)
    bottom_right_fig = create_batted_heatmap(df)

    # Convert plots to images (Equal size for 2x2 grid)
    plot_width = 4 * inch
    plot_height = (doc.height - 60) / 2  # Ensures equal-sized 2x2 grid

    top_left_img = Image(save_plt_as_png(top_left_fig), width=plot_width, height=plot_height)
    top_right_img = Image(save_plt_as_png(top_right_fig), width=plot_width, height=plot_height)
    bottom_left_img = Image(save_plt_as_png(bottom_left_fig), width=plot_width, height=plot_height)
    bottom_right_img = Image(save_plt_as_png(bottom_right_fig), width=plot_width, height=plot_height)

    # ---- Create Legend ----
    sorted_pitches = pitch_stats_df.sort_values(by="Total", ascending=False).index.tolist()

    # Create legend content using colored dots (●)
    legend_data = [["Pitch Types"]]
    for pitch in sorted_pitches:
        color = pitch_colors.get(pitch, "gray")  # Get assigned color or default to gray
        legend_data.append([Paragraph(f'<font color="{color}">●</font>&nbsp;&nbsp;&nbsp;{pitch}', styles["Normal"])])

    # Create left-aligned legend table
    legend_table = Table(legend_data, colWidths=[1.25 * inch])  # Shifted slightly left
    legend_table.setStyle(TableStyle([
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('FONTNAME', (0, 0), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0), (-1, -1), 12),
    ]))

    scatter_plot_layout = Table([
        [legend_table, top_left_img, top_right_img],
        ["", bottom_left_img, bottom_right_img]
    ], colWidths=[1.75 * inch, plot_width, plot_width])

    scatter_plot_layout.setStyle(TableStyle([
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),  # Forces all content to align at the top
        ('ALIGN', (0, 0), (-1, -1), 'LEFT')  # Ensures text alignment in cells
    ]))

    elements.append(scatter_plot_layout)

    # Build PDF
    doc.build(elements)

In [735]:
create_pitcher_report(
    pitcher_name=pitcherName,
    pitch_stats_df=df_stats,
    game_stats_df=statLine,
    df=df,
)